In [49]:
import sympy
from sympy import Matrix, Rational, sqrt, symbols
import numpy as np


# Mathematics for Machine Learning

## Session 04: LU decomposition

## Gerhard Jäger

### October 31, 2024

# LU decomposition

### triangular matrices

- an **upper triangular matrix** is a square matrix with only $0$ below the main diagonal


In [50]:
Matrix([
    [1,1,1],
    [0,1,1],
    [0,0,1]
])

Matrix([
[1, 1, 1],
[0, 1, 1],
[0, 0, 1]])

In [51]:
Matrix([
    [0,1,6],
    [0,1,1],
    [0,0,0]
])

Matrix([
[0, 1, 6],
[0, 1, 1],
[0, 0, 0]])

- a **lower triangular matrix** is a square matrix with only $0$ above the main diagonal

In [52]:
Matrix([
    [0,1,6],
    [0,1,1],
    [0,0,0]
]).T

Matrix([
[0, 0, 0],
[1, 1, 0],
[6, 1, 0]])

- the transpose of an upper triangular matrix is lower triangular, and vice versa
- *Gaussian elimination* transforms a matrix into an upper triangular matrix
- *Jordan elimination* transforms a matrix into a lower triangular matrix
- $\Rightarrow$ Gauss-Jordan elimination produces a diagonal matrix



Let us focus on the Gauss part.

In [53]:
A = Matrix([
    [1,2,1],
    [3,2,4],
    [4,4,3]
])
A

Matrix([
[1, 2, 1],
[3, 2, 4],
[4, 4, 3]])

In [54]:
E1 = Matrix([
    [1,0,0],
    [-3,1,0],
    [0,0,1]
])
E1*A

Matrix([
[1,  2, 1],
[0, -4, 1],
[4,  4, 3]])

In [55]:
E2 = Matrix([
    [1,0,0],
    [0,1,0],
    [-4,0,1]
])
E2 * E1 * A

Matrix([
[1,  2,  1],
[0, -4,  1],
[0, -4, -1]])

In [56]:
E3 = Matrix([
    [1,0,0],
    [0,1,0],
    [0,-1,1]
])
E3 * E2 * E1 * A

Matrix([
[1,  2,  1],
[0, -4,  1],
[0,  0, -2]])

Let us call this upper triangular matrix resulting from Gauss elimination $U$.

$$
\begin{aligned}
U &= \displaystyle \left[\begin{matrix}1 & 2 & 1\\0 & -4 & 1\\0 & 0 & -2\end{matrix}\right]\\
E_3 E_2 E_1 A &= U
\end{aligned}
$$

Assuming that $E_i$ is invertible (which it is), this entails

$$
\begin{aligned}
A &= (E_3 E_2 E_1)^{-1} U
\end{aligned}
$$

### Side remark: inverse of a matrix product

- suppose both $A$ and $B$ are invertible. What is $(AB)^{-1}$?

$$
\begin{aligned}
(AB)^{-1} &= X\\
XAB &= \mathbf I\\
XA &= \mathbf I B^{-1}\\
 &= B^{-1}\\
 X &= B^{-1}A^{-1}
\end{aligned}
$$

$$
\Large
    (AB)^{-1} = B^{-1}A^{-1}
$$

While we're at it: What is the transpose of a matrix product?

$$
\begin{aligned}
(A^TB^T)_{i,j} &= \sum_k a^T_{ik}b^T_{kj}\\
    &= \sum_k a_{ki}b_{jk}\\
    &= (BA)_{ji}\\
    &=((BA)^T)_{ij}
\end{aligned}
$$

Therefore 


$$
\Large
    (AB)^T = B^TA^T
$$

Still, while we're at it, what is the inverse of an transpose?

$$
\begin{aligned}
(A^T)^{-1} &= X\\
A^T X &= \mathbf I\\
X^T A &= \mathbf I\\
X^T &= A^{-1}\\
X &= (A^{-1})^T
\end{aligned}
$$


$$
\Large
   (A^T)^{-1}  = (A^{-1})^T
$$

Back to Gauss elimination.


$$
\begin{aligned}
A &= (E_3 E_2 E_1)^{-1} U\\
&= E_1^{-1} E_2^{-1} E_3^{-1} U
\end{aligned}
$$

In [57]:
E1

Matrix([
[ 1, 0, 0],
[-3, 1, 0],
[ 0, 0, 1]])

In [58]:
E1.inv()

Matrix([
[1, 0, 0],
[3, 1, 0],
[0, 0, 1]])

It is generally true:

- If $A$ is equal to $\mathbf I$ except for one off-diagonal entry, then $A^{-1}$ is like $A$ except that the off-diagonal entry is multiplied with $-1$.
    
**consequence**: all $E_i^{-1}$ are lower triangular

**fact**: The product of two lower-triangular matrices is lower triangular.

**consequences**: 

- There is a lower-triangular matrix $L$ such that 

$$
\begin{aligned}
A &= LU\\
L &= E_1^{-1} \cdots E_n^{-1}
\end{aligned}
$$
where $E_1, \ldots, E_n$ are the elimination matrices corresponding to the steps of Gauss elimination.

Our example:

In [59]:
L = E1.inv() * E2.inv() * E3.inv()
L

Matrix([
[1, 0, 0],
[3, 1, 0],
[4, 1, 1]])

$$
\begin{aligned}
A &= LU\\
\displaystyle \left[\begin{matrix}1 & 2 & 1\\3 & 2 & 4\\4 & 4 & 3\end{matrix}\right]
&=
\displaystyle \left[\begin{matrix}1 & 0 & 0\\3 & 1 & 0\\4 & 1 & 1\end{matrix}\right]
\displaystyle \left[\begin{matrix}1 & 2 & 1\\0 & -4 & 1\\0 & 0 & -2\end{matrix}\right]
\end{aligned}
$$

For each matrix $A$, there are 
- a permutation matrix $P$,
- an upper triangular matrix $U$
- a lower triangular matrix $L$,
such that 

$$
PA = LU
$$

How do we find $L$?


Simple rule: 
- all diagonal entries of $L$ are 1
- if any elimination matrix $E$ used during Gauss elimination has an off-diagonal entry $e_{i,j} = a$, then $l_{i,j} = -a$
- all other entries are 0


### Example

$$
A = \begin{bmatrix}
4 & 3 & 2 & 1\\
3 & 3 & 2 & 1\\
2 & 2 & 2 & 1\\
1 & 1 & 1 & 1
\end{bmatrix}
$$

- initialize $X := \mathbf I$ and $Y := A$

$$
\begin{align}
X && Y\\
\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1
\end{bmatrix} && \begin{bmatrix}
4 & 3 & 2 & 1\\
3 & 3 & 2 & 1\\
2 & 2 & 2 & 1\\
1 & 1 & 1 & 1
\end{bmatrix}
\end{align}
$$

In [60]:
A = Matrix([
    [4,3,2,1],
    [3,3,2,1],
    [2,2,2,1],
    [1,1,1,1]
])
A

Matrix([
[4, 3, 2, 1],
[3, 3, 2, 1],
[2, 2, 2, 1],
[1, 1, 1, 1]])

In [61]:
A.LUdecomposition()[0]

Matrix([
[  1,   0,   0, 0],
[3/4,   1,   0, 0],
[1/2, 2/3,   1, 0],
[1/4, 1/3, 1/2, 1]])

In [62]:
A.LUdecomposition()[1]

Matrix([
[4,   3,   2,   1],
[0, 3/4, 1/2, 1/4],
[0,   0, 2/3, 1/3],
[0,   0,   0, 1/2]])

- perform Gauss elimination on $Y$ and store the elimination coefficients in $X$.

$$
\begin{align}
X && Y\\
\begin{bmatrix}
1 & 0 & 0 & 0\\
\frac{3}{4} & 1 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1
\end{bmatrix} && \begin{bmatrix}
4 & 3 & 2 & 1\\
0 & \frac{3}{4} & \frac{1}{2} & \frac{1}{4}\\
2 & 2 & 2 & 1\\
1 & 1 & 1 & 1
\end{bmatrix}
\end{align}
$$


- perform Gauss elimination on $Y$ and store the elimination coefficients in $X$.

$$
\begin{align}
X && Y\\
\begin{bmatrix}
1 & 0 & 0 & 0\\
\frac{3}{4} & 1 & 0 & 0\\
\frac{1}{2} & 0 & 1 & 0\\
0 & 0 & 0 & 1
\end{bmatrix} && \begin{bmatrix}
4 & 3 & 2 & 1\\
0 & \frac{3}{4} & \frac{1}{2} & \frac{1}{4}\\
0 & \frac{1}{2} & 1 & \frac{1}{2}\\
1 & 1 & 1 & 1
\end{bmatrix}
\end{align}
$$


- perform Gauss elimination on $Y$ and store the elimination coefficients in $X$.

$$
\begin{align}
X && Y\\
\begin{bmatrix}
1 & 0 & 0 & 0\\
\frac{3}{4} & 1 & 0 & 0\\
\frac{1}{2} & 0 & 1 & 0\\
\frac{1}{4} & 0 & 0 & 1
\end{bmatrix} && \begin{bmatrix}
4 & 3 & 2 & 1\\
0 & \frac{3}{4} & \frac{1}{2} & \frac{1}{4}\\
0 & \frac{1}{2} & 1 & \frac{1}{2}\\
0 & \frac{1}{4} & \frac{1}{2} & \frac{3}{4}
\end{bmatrix}
\end{align}
$$


- perform Gauss elimination on $Y$ and store the elimination coefficients in $X$.

$$
\begin{align}
X && Y\\
\begin{bmatrix}
1 & 0 & 0 & 0\\
\frac{3}{4} & 1 & 0 & 0\\
\frac{1}{2} & \frac{2}{3} & 1 & 0\\
\frac{1}{4} & 0 & 0 & 1
\end{bmatrix} && \begin{bmatrix}
4 & 3 & 2 & 1\\
0 & \frac{3}{4} & \frac{1}{2} & \frac{1}{4}\\
0 & 0 &  \frac{2}{3} & \frac{1}{3}\\
0 & \frac{1}{4} & \frac{1}{2} & \frac{3}{4}
\end{bmatrix}
\end{align}
$$


- perform Gauss elimination on $Y$ and store the elimination coefficients in $X$.

$$
\begin{align}
X && Y\\
\begin{bmatrix}
1 & 0 & 0 & 0\\
\frac{3}{4} & 1 & 0 & 0\\
\frac{1}{2} & \frac{2}{3} & 1 & 0\\
\frac{1}{4} & \frac{1}{3} & 0 & 1
\end{bmatrix} && \begin{bmatrix}
4 & 3 & 2 & 1\\
0 & \frac{3}{4} & \frac{1}{2} & \frac{1}{4}\\
0 & 0 & \frac{2}{3} & \frac{1}{3}\\
0 & 0 & \frac{1}{3} & \frac{2}{3}
\end{bmatrix}
\end{align}
$$


- perform Gauss elimination on $Y$ and store the elimination coefficients in $X$.

$$
\begin{align}
X && Y\\
\begin{bmatrix}
1 & 0 & 0 & 0\\
\frac{3}{4} & 1 & 0 & 0\\
\frac{1}{2} & \frac{2}{3} & 1 & 0\\
\frac{1}{4} & \frac{1}{3} & \frac{1}{2} & 1
\end{bmatrix} && \begin{bmatrix}
4 & 3 & 2 & 1\\
0 & \frac{3}{4} & \frac{1}{2} & \frac{1}{4}\\
0 & 0 & \frac{2}{3} & \frac{1}{3}\\
0 & 0 & 0 & \frac{1}{2}
\end{bmatrix}
\end{align}
$$

Finally:
- $L := X$
- $U := Y$

## When is a matrix invertible?

The steps of Jordan elimination do not alter the diagonal. Hence, if, after Gauss elimination, we have no zeros on the diagonal, the rest of Gauss-Jordan will go through and we will find the inverse matrix.

Conversely, this means:

**A square matrix is invertible if and only if Gauss elimination produces an upper triangular matrix with only non-zero entries on the diagonal.**

Example of a non-invertible matrix:

In [63]:
B = Matrix(
[
    [1, -4, 2],
    [-2, 1, 3],
    [2,6,-10]
])
B

Matrix([
[ 1, -4,   2],
[-2,  1,   3],
[ 2,  6, -10]])

In [64]:
E1 = Matrix([
    [1,0,0],
    [2,1,0],
    [0,0,1]
])
E1 * B

Matrix([
[1, -4,   2],
[0, -7,   7],
[2,  6, -10]])

In [65]:
E2 = Matrix([
    [1,0,0],
    [0,1,0],
    [-2,0,1]
])
E2 * E1 * B

Matrix([
[1, -4,   2],
[0, -7,   7],
[0, 14, -14]])

In [66]:
E3 = Matrix([
    [1,0,0],
    [0,1,0],
    [0,2,1]
])
E3 * E2 * E1 * B

Matrix([
[1, -4, 2],
[0, -7, 7],
[0,  0, 0]])

We have a $0$ on the diagonal, therefore $B$ is not invertible.

LU decomposition also works for non-invertible matrices though.

In [67]:
U = E3 * E2 * E1 * B
U

Matrix([
[1, -4, 2],
[0, -7, 7],
[0,  0, 0]])

In [68]:
L = E1.inv() * E2.inv() * E3.inv()
L

Matrix([
[ 1,  0, 0],
[-2,  1, 0],
[ 2, -2, 1]])

In [69]:
L * U

Matrix([
[ 1, -4,   2],
[-2,  1,   3],
[ 2,  6, -10]])

In [70]:
B

Matrix([
[ 1, -4,   2],
[-2,  1,   3],
[ 2,  6, -10]])

## LDU decomposition

L is a lower triangular matrix with all-$1$ on the diagonal

We can factorize U further into a *diagonal matrix* and an upper triangular matrix which also has all-$1$ on the diagonal.

$$
A = LDU'
$$

- if $u_{ii}\neq 0$, we set $d_{ii} = u_{ii}$ and divide the $i$th row of U by this value
- if $u_{ii}= 0$ we set $d_{ii}=0$ and $u'_{ii} = 1$.

$$
\begin{aligned}
\left[\begin{matrix}1 & -4 & 2\\-2 & 1 & 3\\2 & 6 & -10\end{matrix}\right]
&=
\left[\begin{matrix}1 & 0 & 0\\-2 & 1 & 0\\2 & -2 & 1\end{matrix}\right]
\left[\begin{matrix}1 & -4 & 2\\0 & -7 & 7\\0 & 0 & 0\end{matrix}\right]\\[1em]
&=\left[\begin{matrix}1 & 0 & 0\\-2 & 1 & 0\\2 & -2 & 1\end{matrix}\right]
\left[\begin{matrix}1 & 0 & 0\\0 & -7 & 0\\0 & 0 & 0\end{matrix}\right]
\left[\begin{matrix}1 & -4 & 2\\0 & 1 & -1\\0 & 0 & 1\end{matrix}\right]
\end{aligned}
$$

### LU decomposition in SymPy

In [71]:
B

Matrix([
[ 1, -4,   2],
[-2,  1,   3],
[ 2,  6, -10]])

In [72]:
l,u,p = B.LUdecomposition()

In [73]:
l

Matrix([
[ 1,  0, 0],
[-2,  1, 0],
[ 2, -2, 1]])

In [74]:
u

Matrix([
[1, -4, 2],
[0, -7, 7],
[0,  0, 0]])

In [75]:
p

[]

The third component, `p`, contains the permutation required to ensure the success of LU decomposition.



Back to our old example:

In [76]:
A = Matrix([
    [1,4,5],
    [4,16,6],
    [5,6,3]
])
A

Matrix([
[1,  4, 5],
[4, 16, 6],
[5,  6, 3]])

In [77]:
l,u,p = A.LUdecomposition()

In [78]:
l

Matrix([
[1, 0, 0],
[5, 1, 0],
[4, 0, 1]])

In [79]:
u

Matrix([
[1,   4,   5],
[0, -14, -22],
[0,   0, -14]])

In [80]:
p

[[1, 2]]

### LU decomposition with numpy arrays

In [81]:
import numpy as np
from scipy.linalg import lu

In [82]:
A = np.array([
    [1,4,5],
    [4,16,6],
    [5,6,3]
])
A

array([[ 1,  4,  5],
       [ 4, 16,  6],
       [ 5,  6,  3]])

In [83]:
p,l,u = lu(A)

In [84]:
p

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [85]:
l

array([[1.  , 0.  , 0.  ],
       [0.8 , 1.  , 0.  ],
       [0.2 , 0.25, 1.  ]])

In [86]:
u

array([[ 5. ,  6. ,  3. ],
       [ 0. , 11.2,  3.6],
       [ 0. ,  0. ,  3.5]])

In [87]:
p @ l @ u

array([[ 1.,  4.,  5.],
       [ 4., 16.,  6.],
       [ 5.,  6.,  3.]])

### Why should we bother about LU decomposition?

In numerical linear algebra, doing the LU decomposition once often saves a lot of computing time further down the road.

E.g., suppose we have done the LU decomposition of some matrix A.

$$
\begin{aligned}
A &= \left[\begin{array}{r}1 & 2 & 1\\3 & 2 & 4\\4 & 4 & 3\end{array}\right]\\[1em]
L &= \left[\begin{array}{r}1 & 0 & 0\\3 & 1 & 0\\4 & 1 & 1\end{array}\right]\\[1em]
U &= \left[\begin{array}{r}1 & 2 & 1\\0 & -4 & 1\\0 & 0 & -2\end{array}\right]
\end{aligned}
$$

Now suppose you (or rather: the computer) are/is asked to solve the linear system

$$
A\begin{bmatrix}x\\y\\z\end{bmatrix} = \begin{bmatrix}1\\1\\0\end{bmatrix}
$$

Mathematically, an obvious way is

$$
\begin{aligned}
\begin{bmatrix}x\\y\\z\end{bmatrix} &= A^{-1}\begin{bmatrix}1\\1\\0\end{bmatrix}\\[1em]
&= U^{-1}L^{-1}\begin{bmatrix}1\\1\\0\end{bmatrix}
\end{aligned}
$$

This requires us to find the inverses of L and U. However, a more efficient way is the following:

$$
\begin{aligned}
L
\begin{bmatrix}
w\\u\\v
\end{bmatrix} &= \begin{bmatrix}1\\1\\0\end{bmatrix}\\[1em]
U\begin{bmatrix}x\\y\\z\end{bmatrix} &= \begin{bmatrix}
w\\u\\v
\end{bmatrix}
\end{aligned}
$$

Reason: We can solve these systems via **substitution**, without elimination.
    
$$
\begin{aligned}
\left[\begin{array}{r}
1 & 0 & 0
\\3 & 1 & 0
\\4 & 1 & 1
\end{array}\right]
\begin{bmatrix}
w\\u\\v
\end{bmatrix}
&= \begin{bmatrix}1\\1\\0\end{bmatrix}\\[1em]
\end{aligned}
$$

In non-matrix notation:

$$
\begin{aligned}
w & & &= 1\\
3w &+ u &&= 1\\
4w &+u &+v &= 0
\end{aligned}
$$

- substituting $w$:

$$
\begin{aligned}
3\cdot 1 &+ u &&= 1\\
4\cdot 1 &+u &+v &= 0
\end{aligned}
$$

- constants to the left-hand side

$$
\begin{aligned}
 u &&= -2\\
u &+v &= -4
\end{aligned}
$$

- same with $v$

$$
\begin{aligned}
-2 &+v &= -4\\
&v&= -2\\[1em]
\begin{bmatrix}w\\u\\v\end{bmatrix} &= 
\begin{bmatrix}
1\\-2\\-2
\end{bmatrix}
\end{aligned}
$$

next system of equations.

$$
\left[\begin{array}{r}1 & 2 & 1\\0 & -4 & 1\\0 & 0 & -2\end{array}\right]
\begin{bmatrix}
x\\y\\z
\end{bmatrix} = 
\begin{bmatrix}
1\\-2\\-2
\end{bmatrix}\\[1em]
$$

$$
\begin{aligned}
&z &&= 1\\\hline
x & +2y  &&= 0\\
  & -4y &+ z& = -3\\\hline
  y &&&= \frac{3}{4}\\\hline
  x &&&=-\frac{3}{2}\\\hline
\end{aligned}
$$

$$
\begin{aligned}
   \begin{bmatrix}
   x\\y\\z
   \end{bmatrix} 
   &= 
   \left[
   \begin{array}{r}
   -\frac{3}{2}\\
   \frac{3}{4}\\
   1
   \end{array}\right]
\end{aligned}
$$

## Symmetric matrices

- symmetric matrices are square matrices $S$ with the property that

$$
S = S^T
$$

- if there is an LDU decomposition for a symmetric matrix $S$, then

$$
\begin{aligned}
S &= LDU\\
L &= U^T
\end{aligned}
$$

- in other words, a symmetric matrix $S$ can be decomposed as
$$
S = LDL^T
$$

- if row permutation is required, it has to be accompanied by column permutation to preserve symmetry

### example



In [88]:
S = Matrix([
    [0, 1, 2],
    [1, -1, 1],
    [2, 1, 3]
])
S

Matrix([
[0,  1, 2],
[1, -1, 1],
[2,  1, 3]])

- permutation matrix

In [89]:
l, u, p = S.LUdecomposition()

In [90]:
l

Matrix([
[1, 0, 0],
[0, 1, 0],
[2, 3, 1]])

In [91]:
u

Matrix([
[1, -1,  1],
[0,  1,  2],
[0,  0, -5]])

In [92]:
p

[[0, 1]]

In [93]:
l * u

Matrix([
[1, -1, 1],
[0,  1, 2],
[2,  1, 3]])

In [94]:
P = Matrix([
    [0,0,1],
    [0,1,0],
    [1,0,0]
])
P

Matrix([
[0, 0, 1],
[0, 1, 0],
[1, 0, 0]])

In [95]:
S1 = P * S * P.T
S1

Matrix([
[3,  1, 2],
[1, -1, 1],
[2,  1, 0]])

$$
\begin{aligned}
    L &= \left[\begin{matrix}1 & 0 & 0\\\frac{1}{3} & 1 & 0\\\frac{2}{3} & - \frac{1}{4} & 1\end{matrix}\right]\\[1em]
    D &= \begin{bmatrix}
    3 & 0 & 0\\
    0 & -\frac{4}{3} & 0\\
    0 & 0 & -\frac{5}{4}
    \end{bmatrix}\\
    PSP^T &= L D L^T
\end{aligned}
$$
